# モデル改善の試行錯誤用ノートブック  

# 価格予測モデルのBaseline  
- CNNを用いたモデルを作成する．  
- 価格予測とクラス分類でタスクが大きく異なるので，imagenetで学習したモデルを用いないものを最初に作成する．  
- サイトに載せられる画像を教師データとしており，画像が大きく回転したりなどは不要と考えられるためそのような前処理は行わない．  
- 損失関数にはmaeもしくはrmseを用いる．  

## モデルの構築  
- EfficientNetB0（未学習）を用いて特徴量を抽出．  
- num_sales, コレクション名のone-hotベクトルを抽出した特徴量に結合．  
- 全結合層を重ねて出力．  
- ImageNetを用いて事前学習したものとしていないもので比較する．  
- 目的変数をそのまま予測するとスケールが大きすぎるので，先に対数変換して評価関数にRMSE, MAEなどを用いるほうが良いかも．  
- **このノートブックでやっているのは事前学習有り．**  

## 評価関数  
- RMSLEを用いる．  
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n (\log{(y_i+1)} - \log{(\hat{y_i} +1)})^2}$$

- 追加でMAPEを用いてみる．  
$$MAPE = \frac{100}{n} \sum_{i=1}^n |\frac{\hat{y}_i - y_i}{y_i}|$$

タスクAに関してはデータ不足の可能性が考えられるため，特徴量抽出とともにデータを追加で収集する．  

## 変数（タスクA）  
- 目的変数: last_sale.total_price  
- 説明変数: 画像データ，コレクション名（collection.name），num_sales，

## 変数（タスクB）  
- 目的変数: last_sale.total_price  
- 説明変数: 画像データ  

### 実験結果  
- 2021/11/16: ベースモデルをSwinTransformerにして学習
    - SwinTransformerのGit: https://github.com/rishigami/Swin-Transformer-TF
    - Apach Lisence 2.0のため商用利用可能  

In [1]:
import os
from typing import List, Optional, Tuple, Dict
import math
import tempfile
import random
import sys

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import cv2
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optim
import tensorflow.keras.activations as activations
from tensorflow.keras.utils import Sequence
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.applications import EfficientNetB0 as efn
import cloudpickle

sys.path.append("../Swin-Transformer-TF")
from swintransformer import SwinTransformer

In [2]:
A_IMGPATH = "../data/taskA/img"
A_DFPATH = "../data/taskA/table"
B_IMGPATH = "../data/taskB/img"
B_DFPATH = "../data/taskB/table"
asset_df_A = pd.read_csv(os.path.join(A_DFPATH, "asset_data.csv"))
asset_df_B = pd.read_csv(os.path.join(B_DFPATH, "asset_data.csv"))

asset_df_A = asset_df_A.rename(columns={"last_sale.total_price": "target"})
asset_df_B = asset_df_B.rename(columns={"last_sale.total_price": "target"})

asset_df_A = pd.concat((asset_df_A, pd.get_dummies(asset_df_A["collection.name"])), axis=1)
asset_df_B[asset_df_A["collection.name"].unique()] = 0

asset_df_A["full_path"] =\
    asset_df_A["image_id"].apply(lambda x: A_IMGPATH + "/" + x)
asset_df_B["full_path"] =\
    asset_df_B["image_id"].apply(lambda x: B_IMGPATH + "/" + x)

asset_df_A['target'] = asset_df_A['target'].astype(float) * 1e-18
asset_df_B['target'] = asset_df_B['target'].astype(float) * 1e-18
asset_df_A = asset_df_A.query('target > 0')
asset_df_B = asset_df_B.query('target > 0')
asset_df_A['target'] = asset_df_A['target'].apply(lambda x: np.log1p(x))
asset_df_B['target'] = asset_df_B['target'].apply(lambda x: np.log1p(x))

os.makedirs("../models", exist_ok=True)

print(f"data shape: {asset_df_A.shape}")
print(f"data shape: {asset_df_B.shape}")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,27,28,71,88,119) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


data shape: (21747, 169)
data shape: (7861, 169)


## Helper Functions  

### DataLoader  

In [3]:
class FullPathDataLoader(Sequence):
    """
    Data loader that load images, meta data and targets.
    This class is inherited Sequence class of Keras.
    """

    def __init__(self, path_list: np.ndarray, target: Optional[np.ndarray] = None,
                 meta_data: Optional[np.ndarray] = None, batch_size: int = 16,
                 task: str = "B", width: int = 256, height: int = 256,
                 resize: bool = True, shuffle: bool = True, is_train: bool = True):
        """
        Constructor. This method determines class variables.

        Parameters
        ----------
        path_list : np.ndarray[str]
            The array of absolute paths of images.
        meta_data : np.ndarray[int]
            One-hot vector of collections.
        target : np.ndarray
            Array of target variavles.
        batch_size : int
            Batch size used when model training.
        task : str
            Please determine this data loader will be used for task A or B(default=A).
        width : int
            Width of resized image.
        height : int
            Height of resize image.
        resize : bool
            Flag determine whether to resize.
        shuffle : bool
            Flag determine whether to shuffle on epoch end.
        is_train : bool
            Determine whether this data loader will be used training model.
            if you won't this data loader, you have set 'is_train'=False.
        """
        self.path_list = path_list
        self.batch_size = batch_size
        self.task = task
        self.width = width
        self.height = height
        self.resize = resize
        self.shuffle = shuffle
        self.is_train = is_train
        self.length = math.ceil(len(self.path_list) / self.batch_size)

        if self.is_train:
            self.target = target
        if self.task == "A":
            self.meta_data = meta_data

    def __len__(self):
        """
        Returns
        -------
        self.length : data length
        """
        return self.length

    def get_img(self, path_list: np.ndarray):
        """
        Load image data and resize image if 'resize'=True.

        Parameters
        ----------
        path_liist : np.ndarray
            The array of relative image paths from directory 'dir_name'.
            Size of this array is 'batch_size'.

        Returns
        -------
        img_list : np.ndarray
            The array of image data.
            Size of an image is (width, height, 3) if 'resize'=True.
        '"""
        img_list = []
        for path in path_list:
            img = cv2.imread(path)
            img = cv2.resize(img, (self.width, self.height))
            img = img / 255.
            img_list.append(img)

        img_list = np.array(img_list)
        return img_list

    def _shuffle(self):
        """
        Shuffle path_list, meta model.
        If 'is_train' is True, target is shuffled in association path_list.
        """
        idx = np.random.permutation(len(self.path_list))
        self.path_list = self.path_list[idx]
        if self.task == "A":
            self.meta_data = self.meta_data[idx]
        if self.is_train:
            self.target = self.target[idx]

    def __getitem__(self, idx):
        path_list = self.path_list[self.batch_size*idx:self.batch_size*(idx+1)]
        img_list = self.get_img(path_list)
        if self.is_train:
            target_list = self.target[self.batch_size*idx:self.batch_size*(idx+1)]
            if self.task == "A":
                meta = self.meta_data[self.batch_size*idx:self.batch_size*(idx+1)]
                return (img_list, meta), target_list
            else:
                return img_list, target_list
        else:
            if self.task == "A":
                meta = self.meta_data[self.batch_size*idx:self.batch_size*(idx+1)]
                return ((img_list, meta),)
            else:
                return img_list

    def on_epoch_end(self):
        if self.is_train:
            self._shuffle()

### seed settings  

In [4]:
def set_seed(random_state=6174):
    tf.random.set_seed(random_state)
    np.random.seed(random_state)
    random.seed(random_state)
    os.environ['PYTHONHASHSEED'] = str(random_state)

### Create model  

In [5]:
def create_model(input_shape: Tuple[int], output_shape: int,
                 activation, loss, meta_shape: Optional[int] = None,
                 task: str = "B", learning_rate: float = 0.001,
                 pretrain: bool = False) -> models.Model:
    """
    The function for creating model.

    Parameters
    ----------
    input_shape : int
        Shape of input image data.
    output_shape : int
        Shape of model output.
    activation : function
        The activation function used hidden layers.
    loss : function
        The loss function of model.
    meta_shape : int
        Shape of input meta data of image.
    task : str
        Please determine this model will be used for task A or B(default=A).
    learning_rate : float
        The learning rate of model.
    pretrain : bool
        Flag that deterimine whether use pretrain model(default=False).

    Returns
    -------
    model : keras.models.Model
        Model instance.
    """
    if pretrain:
        weights = 'imagenet'
    else:
        weights = None

    inputs = layers.Input(shape=input_shape)
    base_model = SwinTransformer('swin_tiny_224', include_top=False, pretrained=True, use_tpu=False)(inputs)

    if task == "A":
        meta_inputs = layers.Input(shape=meta_shape)
        concate = layers.Concatenate()([base_model, meta_inputs])
        dense1 = layers.Dense(units=128)(concate)
        av1 = layers.Activation(activation)(dense1)
        dr1 = layers.Dropout(0.3)(av1)
        dense2 = layers.Dense(units=64)(dr1)
        av2 = layers.Activation(activation)(dense2)
        dr2 = layers.Dropout(0.3)(av2)
        outputs = layers.Dense(output_shape)(dr2)

        model = models.Model(inputs=[inputs, meta_inputs], outputs=[outputs])

    elif task == "B":
        dense1 = layers.Dense(units=128)(base_model)
        av1 = layers.Activation(activation)(dense1)
        dr1 = layers.Dropout(0.3)(av1)
        dense2 = layers.Dense(units=64)(dr1)
        av2 = layers.Activation(activation)(dense2)
        dr2 = layers.Dropout(0.3)(av2)
        outputs = layers.Dense(output_shape)(dr2)

        model = models.Model(inputs=[inputs], outputs=[outputs])

    else:
        raise Exception("Please set task is A or B.")

    model.compile(loss=loss,
                  optimizer=optim.Adam(learning_rate=learning_rate),
                  metrics=['mae', 'mse'])
    return model

### Training model  

In [6]:
def train(path_list: np.ndarray, target: np.ndarray, loss,
          meta_data: Optional[np.ndarray] = None, task: str = "B"):
    """
    The function for training model.

    Parameters
    ----------
    path_list : np.ndarray
        The path list of all image data.
    target : np.ndarray
        The array of targets data.
    loss : function
        The loss function of keras.
    meta_data : np.ndarray
        The array of meta data of image.
    task : str
        Please determine you train model for task A or B(default=A).
    """
    if task == "A":
        train_path, val_path, train_meta, val_meta, train_y, val_y =\
            train_test_split(path_list, meta_data, target, test_size=0.1, random_state=6174)
        train_gen = FullPathDataLoader(path_list=train_path, target=train_y,
                                       meta_data=train_meta, batch_size=16,
                                       width=224, height=224, task=task)
        val_gen = FullPathDataLoader(path_list=val_path, target=train_y,
                                     meta_data=val_meta, batch_size=1,
                                     width=224, height=224, task=task)
    elif task == "B":
        train_path, val_path, train_y, val_y =\
            train_test_split(path_list, target, test_size=0.1, random_state=6174)
        train_gen = FullPathDataLoader(path_list=train_path, target=train_y,
                                       width=224, height=224, batch_size=16, task=task)
        val_gen = FullPathDataLoader(path_list=val_path, target=val_y,
                                     width=224, height=224, batch_size=1, task=task)
    else:
        raise Exception("Please set task is A or B")

    set_seed()
    model = NFTModel(
        model_func=create_model, input_shape=(224, 224, 3),
        output_shape=1,activation=activations.relu, loss=loss,
        meta_shape=len(meta_features), task=task,
        learning_rate=0.00001, pretrain=True
    )

    ES = callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                 restore_best_weights=True)

    print("starting training")
    print('*' + '-' * 30 + '*')

    model.fit(train_gen, val_gen, epochs=100, batch_size=16,
              callbacks=[ES])

    print("finished training")
    print('*' + '-' * 30 + '*' + '\n')

    if task == "A":
        val_gen = FullPathDataLoader(path_list=val_path, target=train_y,
                                     meta_data=val_meta, batch_size=1, task=task,
                                     width=224, height=224, shuffle=False, is_train=False)
    else:
        val_gen = FullPathDataLoader(path_list=val_path, target=train_y,
                                     batch_size=1, task=task,
                                     width=224, height=224, shuffle=False, is_train=False)
    print("starting evaluate")
    print('*' + '-' * 30 + '*')

    model.evaluate(val_gen, val_y)

    print("finished evaluate")
    print('*' + '-' * 30 + '*' + '\n')

    return model

In [7]:
class NFTModel(KerasRegressor):
    """
    Model class.
    This class is inherited KerasRegressor class of keras.
    """

    def __init__(self, model_func, input_shape, output_shape,
                 activation, loss, meta_shape, task, learning_rate, pretrain):
        """
        Constructor.

        Prameters
        ---------
        model_func : function
            The function for creating model.
        """
        self.model_func = model_func
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.activation = activation
        self.loss = loss
        self.meta_shape = meta_shape
        self.task = task
        self.learning_rate = learning_rate
        self.pretrain = pretrain
        super().__init__(
            build_fn=model_func(input_shape, output_shape,
                                activation=activation, loss=loss,
                                meta_shape=meta_shape, task=task,
                                learning_rate=learning_rate, pretrain=pretrain)
        )
        self.model = self.build_fn

    def __getstate__(self):
        result = {'sk_params': self.sk_params,
                  'model_func': self.model_func,
                  'input_shape': self.input_shape,
                  'output_shape': self.output_shape,
                  'activation': self.activation,
                  'loss': self.loss,
                  'meta_shape': self.meta_shape,
                  'task': self.task,
                  'learning_rate': self.learning_rate,
                  'pretrain': self.pretrain}
        with tempfile.TemporaryDirectory() as dir:
            if hasattr(self, 'model'):
                self.model.save_weights(dir + '/output.h5')
                with open(dir + '/output.h5', 'rb') as f:
                    result['weights'] = f.read()
        return result

    def __setstate__(self, serialized):
        self.sk_params = serialized['sk_params']
        self.model_func = serialized['model_func']
        self.input_shape = serialized['input_shape']
        self.output_shape = serialized['output_shape']
        self.activation = serialized['activation']
        self.loss = serialized['loss']
        self.meta_shape = serialized['meta_shape']
        self.task = serialized['task']
        self.learning_rate = serialized['learning_rate']
        self.pretrain = serialized['pretrain']
        self.model = self.model_func(
                    self.input_shape, self.output_shape,
                    activation=self.activation, loss=self.loss,
                    meta_shape=self.meta_shape, task=self.task,
                    learning_rate=self.learning_rate, pretrain=self.pretrain
                )

        with tempfile.TemporaryDirectory() as dir:
            weight_data = serialized.get('weights')
            if weight_data:
                with open(dir + '/input.h5', 'wb') as f:
                    f.write(weight_data)
                self.model.load_weights(dir + '/input.h5')

    def fit(self, train_gen, val_gen, epochs, batch_size, callbacks=None):
        """
        Training model.

        Parameters
        ----------
        train_gen : iterator
            The generator of train data.
        val_gen : iterator
            The generator of validation data.
        epochs : int
            Number of epochs for training model.
        batch_size : int
            Size of batch for training model.
        callbacks : list
            The list of callbacks.
            For example [EarlyStopping instance, ModelCheckpoint instance]
        """
        self.model.fit(train_gen, epochs=epochs, batch_size=batch_size,
                       validation_data=val_gen, callbacks=callbacks)

    def evaluate(self, test_X, test_y):
        """
        Evaluate model.

        Parameters
        ----------
        test_X : iterator
            The generator of test data.
        test_y : np.ndarray
            The array of targets of test data.
        """
        pred = self.model.predict(test_X)
        pred = np.where(pred < 0, 0, pred)
        rmse = np.sqrt(mean_squared_error(test_y, pred))
        mae = np.sqrt(mean_absolute_error(test_y, pred))

        print(f"RMSE Score: {rmse}")
        print(f"MAE Score: {mae}")

    def predict(self, img_path: str, collection_name: str, num_sales: int,
                task: str = "B"):
        """
        Predict data using trained model.

        Parameters
        ----------
        img_path : str
            The path of image data.
        collection_name : str
            Name of collection of the NFT.
        num_sales : int
            Number of times the NFT sold.
        """
        if task == "A":
            collections = ['CryptoPunks',
                           'Bored Ape Yacht Club',
                           'Edifice by Ben Kovach',
                           'Mutant Ape Yacht Club',
                           'The Sandbox',
                           'Divine Anarchy',
                           'Cosmic Labs',
                           'Parallel Alpha',
                           'Art Wars | AW',
                           'Neo Tokyo Identities',
                           'Neo Tokyo Part 2 Vault Cards',
                           'Cool Cats NFT',
                           'CrypToadz by GREMPLIN',
                           'BearXLabs',
                           'Desperate ApeWives',
                           'Decentraland',
                           'Neo Tokyo Part 3 Item Caches',
                           'Doodles',
                           'The Doge Pound',
                           'Playboy Rabbitars Official',
                           'THE SHIBOSHIS',
                           'THE REAL GOAT SOCIETY',
                           'Sipherian Flash',
                           'Party Ape | Billionaire Club',
                           'Treeverse',
                           'Angry Apes United',
                           'CyberKongz',
                           'Emblem Vault [Ethereum]',
                           'Fat Ape Club',
                           'VeeFriends',
                           'JUNGLE FREAKS BY TROSLEY',
                           'Meebits',
                           'Furballs.com Official',
                           'Kaiju Kingz',
                           'Bears Deluxe',
                           'PUNKS Comic',
                           'Hor1zon Troopers',
                           'Lazy Lions',
                           'LOSTPOETS',
                           'Chain Runners',
                           'Chromie Squiggle by Snowfro',
                           'MekaVerse',
                           'Vox Collectibles',
                           'MutantCats',
                           'World of Women',
                           'SuperFarm Genesis Series',
                           'Eponym by ART AI',]
            collection_dict = {
                 collections[i]: i for i in range(len(collections))
            }
            meta_data = np.zeros(shape=(len(collection_dict)+1))
            if collection_name in collection_dict.keys():
                meta_data[collection_dict[collection_name]] = 1
            meta_data[-1] = num_sales
            meta_data = meta_data.reshape(1, -1)

            img = cv2.resize(cv2.imread(img_path)/255., (224, 224))
            img = img.reshape(1, 224, 224, 3)

            pred = self.model.predict([img, meta_data])
        elif task == "B":
            img = cv2.resize(cv2.imread(img_path)/255., (224, 224))
            img = img.reshape(1, 224, 224, 3)

            pred = self.model.predict(img)
        else:
            raise Exception("Please set task is A or B")

        return pred[0][0]

In [8]:
def save_model(instance, file_name: str):
    """
    Save model as pickle file

    Parameters
    ----------
    instance : Class instance
        The class instance you want to save as pickle file.
    file_name : str
        The absolute path of file saved the instance.
    """
    with open(file_name, mode='wb') as f:
        cloudpickle.dump(instance, f)

In [9]:
def load_model(file_name: str):
    """
    Load the model file of pickle.

    Parameters
    ----------
    file_name : str
        The absolute path of the model file.

    Returns
    -------
    model : tf.keras.models.Model
        Trained model object.
    """
    with open(file_name, mode='rb') as f:
        model = cloudpickle.load(f)

    return model

## Training models  

### TaskA  

In [10]:
meta_features =\
    asset_df_A['collection.name'].unique().tolist() + ['num_sales']

path_list = asset_df_A['full_path'].values
meta_data = asset_df_A[meta_features].values
target = asset_df_A['target'].values

model_A = train(path_list, target, losses.mean_squared_error, meta_data,
                task="A")
# save_model(model_A, "../models/baselineA.pkl")

2021-11-19 08:55:06.711895: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 08:55:06.716515: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 08:55:06.716917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 08:55:06.717585: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

starting training
*------------------------------*
Epoch 1/100


2021-11-19 08:55:10.560778: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-19 08:55:19.483270: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2021-11-19 08:55:21.671193: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1224/1224 [==============================] - 341s 269ms/step - loss: 5.7204 - mae: 0.8646 - mse: 5.7204 - val_loss: 97.2260 - val_mae: 1.3864 - val_mse: 97.2260
Epoch 2/100
1224/1224 [==============================] - 327s 267ms/step - loss: 2.8578 - mae: 0.6891 - mse: 2.8578 - val_loss: 90.9288 - val_mae: 1.4694 - val_mse: 90.9288
Epoch 3/100
1224/1224 [==============================] - 332s 272ms/step - loss: 8.3696 - mae: 0.6471 - mse: 8.3696 - val_loss: 54.6650 - val_mae: 1.4089 - val_mse: 54.6650
Epoch 4/100
1224/1224 [==============================] - 328s 268ms/step - loss: 5.6578 - mae: 0.6362 - mse: 5.6578 - val_loss: 43.6177 - val_mae: 1.4477 - val_mse: 43.6177
Epoch 5/100
1224/1224 [==============================] - 331s 271ms/step - loss: 6.4166 - mae: 0.6300 - mse: 6.4166 - val_loss: 32.1879 - val_mae: 1.3919 - val_mse: 32.1879
Epoch 6/100
1224/1224 [==============================] - 335s 274ms/step - loss: 3.1106 - mae: 0.5585 - mse: 3.1106 - val_loss: 28.0191 - val_mae: 

### TaskA（画像のみ）  

In [11]:
path_list = asset_df_A['full_path'].values
target = asset_df_A['target'].values

model_A = train(path_list, target, losses.mean_squared_error,
                task="B")
save_model(model_A, "../models/swintransformerA.pkl")

starting training
*------------------------------*
Epoch 1/100
1224/1224 [==============================] - 348s 277ms/step - loss: 1.1644 - mae: 0.6134 - mse: 1.1644 - val_loss: 0.7221 - val_mae: 0.4215 - val_mse: 0.7221
Epoch 2/100
1224/1224 [==============================] - 339s 277ms/step - loss: 0.9161 - mae: 0.4922 - mse: 0.9161 - val_loss: 0.6481 - val_mae: 0.3312 - val_mse: 0.6481
Epoch 3/100
1224/1224 [==============================] - 339s 277ms/step - loss: 0.7910 - mae: 0.4399 - mse: 0.7910 - val_loss: 0.6369 - val_mae: 0.3201 - val_mse: 0.6369
Epoch 4/100
1224/1224 [==============================] - 341s 279ms/step - loss: 0.7418 - mae: 0.4188 - mse: 0.7418 - val_loss: 0.6122 - val_mae: 0.3017 - val_mse: 0.6122
Epoch 5/100
1224/1224 [==============================] - 339s 277ms/step - loss: 0.7152 - mae: 0.4010 - mse: 0.7152 - val_loss: 0.6544 - val_mae: 0.3208 - val_mse: 0.6544
Epoch 6/100
1224/1224 [==============================] - 340s 277ms/step - loss: 0.6530 - mae:

### TaskB

In [12]:
path_list = asset_df_B['full_path'].values
target = asset_df_B['target'].values

model_B = train(path_list, target, losses.mean_squared_error)
save_model(model_B, "../models/swintransforfmerB.pkl")

starting training
*------------------------------*
Epoch 1/100
443/443 [==============================] - 122s 258ms/step - loss: 0.2787 - mae: 0.2877 - mse: 0.2787 - val_loss: 0.1290 - val_mae: 0.2057 - val_mse: 0.1290
Epoch 2/100
443/443 [==============================] - 113s 255ms/step - loss: 0.2164 - mae: 0.2425 - mse: 0.2164 - val_loss: 0.1144 - val_mae: 0.1936 - val_mse: 0.1144
Epoch 3/100
443/443 [==============================] - 112s 254ms/step - loss: 0.1755 - mae: 0.2271 - mse: 0.1755 - val_loss: 0.1472 - val_mae: 0.2026 - val_mse: 0.1472
Epoch 4/100
443/443 [==============================] - 112s 254ms/step - loss: 0.1737 - mae: 0.2212 - mse: 0.1737 - val_loss: 0.1077 - val_mae: 0.1917 - val_mse: 0.1077
Epoch 5/100
443/443 [==============================] - 112s 254ms/step - loss: 0.1627 - mae: 0.2110 - mse: 0.1627 - val_loss: 0.1098 - val_mae: 0.1744 - val_mse: 0.1098
Epoch 6/100
443/443 [==============================] - 112s 254ms/step - loss: 0.1300 - mae: 0.2026 - ms

## Predict  

### TaskA  

In [13]:
path_list = asset_df_A['full_path'].values
target = asset_df_A['target'].values

gen = FullPathDataLoader(path_list=path_list, target=target,
                             batch_size=1, shuffle=False, is_train=False,
                             task="B", width=224, height=224)

pred = model_A.model.predict(gen)
asset_df_A['predict'] = pred

### TaskB  

In [14]:
path_list = asset_df_B['full_path'].values
target = asset_df_B['target'].values

gen = FullPathDataLoader(path_list=path_list, target=target,
                             batch_size=1, shuffle=False, is_train=False,
                             task="B", width=224, height=224)

pred = model_B.model.predict(gen)
asset_df_B['predict'] = pred

## Evaluate model  

### Task A

### Task B